## Отчет по практической задаче № 2
1. Описание кейса.

Задачей проекта является разработка системы, позволяющей анализировать исходные данные, собирать данные с веб ресурсов путем парсинга, обрабатывать и сохранять их в базу данных.

2.	Бизнес-процесс – BPMN-нотация 
![alt text](/images/Processing.png)

3.	Архитектура

Архитектура пайплайна представляет собой последовательную обработку исходных файлов формата (excel) с последующей записью данных в базу (SqlLight). Весь код обработки данных написан на языке python и sql. Раз в день запускается скрипт и в базу добавляются новые данные из нового файла.  


### Создаем подключение к базе данных

In [3]:
import pandas as pd
from sqlalchemy import create_engine

# Создание подключения к базе данных SQLite
engine = create_engine('sqlite:///mydatabase.db') 

### Загрузка исходных данных из файла Excel

In [2]:
# Загрузка исходных данных из файла Excel
raw_data = pd.read_excel('C:\\Users\\clic\\OneDrive\\Рабочий стол\\Study\\MGTU2\\data2.xlsx')


### Добавляем префикс к именам столбцов

In [4]:
# Добавляем префикс к именам столбцов
prefix = 'dwh_'
raw_data.columns = [prefix + col for col in raw_data.columns]

### Сохраняем данные в стейджинговой таблице

In [5]:
# Сохраняем данные в стейджинговой таблице
staging_table_name = 'staging_table'
raw_data.to_sql(staging_table_name, engine, if_exists='replace', index=False)

20000

### Читаем данные из таблицы

In [6]:
# Читаем данные из таблицы
result_data = pd.read_sql(f'SELECT * FROM {staging_table_name}', con=engine)

print(result_data)

       dwh_transaction_id  dwh_product_id  dwh_customer_id  \
0                       1               2             2950   
1                       2               3             3120   
2                       3              37              402   
3                       4              88             3135   
4                       5              78              787   
...                   ...             ...              ...   
19995               19996              51             1018   
19996               19997              41              127   
19997               19998              87             2284   
19998               19999               6             2764   
19999               20000              11             1144   

             dwh_transaction_date  dwh_online_order dwh_order_status  \
0      2017-02-25 00:00:00.000000               0.0         Approved   
1      2017-05-21 00:00:00.000000               1.0         Approved   
2      2017-10-16 00:00:00.000000      

 ## Этот парсер выполняет следующие шаги:

* Отправляет GET-запрос на страницу ЦБ, содержащую курсы валют.
* Использует библиотеку BeautifulSoup для парсинга HTML-кода и извлечения данных из таблицы с курсами валют.
* Создает базу данных SQLite и таблицу для хранения данных о курсах валют.
* Вставляет данные в базу данных для каждой валюты, извлеченной из таблицы.


Можно запускать этот скрипт периодически, чтобы обновлять данные в вашей базе данных с последними курсами валют Центрального Банка.

In [1]:
import requests
import sqlite3
from bs4 import BeautifulSoup

# URL Центрального Банка для курса валют
url = 'https://www.cbr.ru/currency_base/daily/'

# Создаем базу данных SQLite
conn = sqlite3.connect('currency_exchange.db')
cursor = conn.cursor()

# Создаем таблицу для хранения данных
cursor.execute('''
    CREATE TABLE IF NOT EXISTS currency_rates (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        currency_name TEXT,
        currency_code TEXT,
        exchange_rate REAL
    )
''')

def update_currency_data():
    # Отправляем GET-запрос и получаем HTML-страницу
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Находим таблицу с курсами валют
        table = soup.find('table', {'class': 'data'})

        # Извлекаем данные и сохраняем их в базу данных
        rows = table.find_all('tr')
        for row in rows[1:]:  # Пропустить заголовок таблицы
            columns = row.find_all('td')
            currency_name = columns[3].text
            currency_code = columns[1].text
            exchange_rate = float(columns[4].text.replace(',', '.'))

            # Вставляем данные в базу данных
            cursor.execute('''
                INSERT INTO currency_rates (currency_name, currency_code, exchange_rate)
                VALUES (?, ?, ?)
            ''', (currency_name, currency_code, exchange_rate))

        # Сохраняем изменения и закрываем соединение с базой данных
        conn.commit()
    else:
        print('Не удалось получить данные с Центрального Банка')

def view_currency_data():
    cursor.execute('SELECT * FROM currency_rates')
    data = cursor.fetchall()
    for row in data:
        print(f'ID: {row[0]}, Название валюты: {row[1]}, Код валюты: {row[2]}, Курс: {row[3]}')

# Функция для обновления данных и просмотра базы данных
def main():
    update_currency_data()
    view_currency_data()

if __name__ == '__main__':
    main()

# Закрываем соединение с базой данных
conn.close()


ID: 1, Название валюты: Австралийский доллар, Код валюты: AUD, Курс: 59.8543
ID: 2, Название валюты: Азербайджанский манат, Код валюты: AZN, Курс: 54.8076
ID: 3, Название валюты: Армянских драмов, Код валюты: AMD, Курс: 23.1561
ID: 4, Название валюты: Белорусский рубль, Код валюты: BYN, Курс: 29.1093
ID: 5, Название валюты: Болгарский лев, Код валюты: BGN, Курс: 50.1956
ID: 6, Название валюты: Бразильский реал, Код валюты: BRL, Курс: 18.5637
ID: 7, Название валюты: Венгерских форинтов, Код валюты: HUF, Курс: 25.8764
ID: 8, Название валюты: Вон Республики Корея, Код валюты: KRW, Курс: 69.3819
ID: 9, Название валюты: Вьетнамских донгов, Код валюты: VND, Курс: 38.6626
ID: 10, Название валюты: Гонконгский доллар, Код валюты: HKD, Курс: 11.9269
ID: 11, Название валюты: Грузинский лари, Код валюты: GEL, Курс: 34.3191
ID: 12, Название валюты: Датская крона, Код валюты: DKK, Курс: 13.1526
ID: 13, Название валюты: Дирхам ОАЭ, Код валюты: AED, Курс: 25.367
ID: 14, Название валюты: Доллар США, Ко